# Implementing our COVID-19 face mask detector for images with OpenCV

Imports Libraries

In [1]:
import os
import cv2
import argparse
import numpy as np

from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

Argument Parser(command line arguments)

In [8]:
ap=argparse.ArgumentParser()

ap.add_argument("--image",required=True,help="path of the input image")
ap.add_argument("--face",required=True,
                help="path to face detector model directory")
ap.add_argument("--prototxt",required=True,help="path to prototxt")
ap.add_argument("--model",type=str,default="mask_detector.model",
                help="path to train mask detector model")

args=vars(ap.parse_args(args=["--image",r"C:\Users\SRKT\Downloads\img1_fb.jpg",
                             "--face",r"C:\Users\SRKT\Desktop\deep-learning-face-detection\res10_300x300_ssd_iter_140000.caffemodel",
                             "--prototxt",r"C:\Users\SRKT\Desktop\deep-learning-face-detection\deploy.prototxt.txt",
                             "--model",r"C:\Users\SRKT\Desktop\face-mask-detector\model.h5"]))


Load Model 

In [9]:
#face detector model
net=cv2.dnn.readNet(args["prototxt"],args["face"])

#face mask detector model
model=load_model(args["model"])

load and pre-process an input image:

In [10]:
image=cv2.imread(args["image"])
orig=image.copy()
image=cv2.resize(image,(600,800))
(h,w)=image.shape[:2]


In [11]:
#construct the blob from image
blob=cv2.dnn.blobFromImage(image,1.0,(300,300),(104.0,177.0,123.0))

#pass the blob through network
print("[INFO] computing face detections....")
net.setInput(blob)
detection=net.forward()

[INFO] computing face detections....


[,frame,no of detections,[class id,class score,conf,x,y,h,w]]

In [12]:
#loop over the detections
for i in range(0,detection.shape[2]):
    #extract the confidence (i.e.,probality) associate with the detection
    confidence=detection[0,0,i,2]
    
    #filter out the weak detections by ensuring the confidence is greater then min confi=0.5
    if confidence>0.5:
         #compute the (x,y) co-ordinate of the bounding box for the object
            box=detection[0,0,i,3:7]*np.array([w,h,w,h])
            (start_x,start_y,end_x,end_y)=box.astype("int")
            
            #ensure the bounding box fall within the dimensions of frame
            (start_x,start_y)=(max(0,start_x),max(0,start_y))
            (end_x,end_y)=(min(w-1,end_x),min(h-1,end_y))
            
            #extract the face ROI and convert it into BGR to RGB
            face=image[start_y:end_y,start_x:end_x]
            face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
            face=cv2.resize(face,(224,224))
            face=img_to_array(face)
            face=preprocess_input(face)
            face=np.expand_dims(face,axis=0)
            
            #pass the face through the model if the face has mask or not
            (mask,withoutMask)=model.predict(face)[0]
            
            #determine  the class label and color  we will draw bounding box and text.
            label="Mask" if mask>withoutMask else "No Mask"
            color=(0,255,0) if label=="Mask" else (0,0,255)
            
            #include the probility in the label
            label="{}: {:.2f}%".format(label,max(mask,withoutMask)*100)
            #display the bounding box rectangle on the output frame
            cv2.putText(image,label,(start_x,start_y-10),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,0,0),2)
            cv2.rectangle(image,(start_x,start_y),(end_x,end_y),color,2)
            

show image output

In [13]:
cv2.imshow("image",image)
cv2.waitKey(0)

-1